In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

import numpy as np
import torchvision
from torchvision import models, transforms
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import time
import os
import copy

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Задача

1. Обучите свою маленькую сверточную сеть с нуля на MNIST или FashionMNIST
2. Поставьте модель в eval()
3. Создайте тензор картинки, которую нужно будет оптимизировать.
4. Задайте функцию потерь.
5. Допишите функцию, которая оптимизирует картинку под вашу функцию потерь и выдаёт её.

Чуть подробнее ниже.

In [9]:
# Вот так можно создать обучаемый тензор со случайными числами и с размерностью картинки 1*28*28
image = torch.randn((1,28,28))*0.001
image = image.to(device)
image.requires_grad = True

На этом этапе у вас уже есть обученная модель с фиксированными параметрами (вы не хотите их больше обучать).

В данном примере в качестве "модели" я возьму пару линейных слоёв.

In [26]:
model = torch.nn.Sequential(
torch.nn.Linear(in_features=1*28*28, out_features=50),
torch.nn.Tanh(),
torch.nn.Linear(in_features=50, out_features=3),
torch.nn.Softmax(dim=1)
)

In [27]:
# так мы переводим модель в режим inference [вывод предсказаний],
# это особенно важно, если в вашей моделе есть такие слои как Dropout/Batchnorm
# потому что в процессе оптимизации картинки вы не хотите чтобы параметры сети менялись
model.eval()

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): Tanh()
  (2): Linear(in_features=50, out_features=3, bias=True)
  (3): Softmax(dim=1)
)

In [29]:
# быстрая проверка что сейчас выдаёт модель на картинку
model.to(device)
image.to(device)

# вам не нужно будет делать flatten в случае сверточной модели
output = model(image.flatten().unsqueeze(0))
output

tensor([[0.3431, 0.3402, 0.3167]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [30]:
torch.argmax(output)

tensor(0, device='cuda:0')

Теперь необходимо дописать вот эту функцию, которая принимает на вход номер класса (**class_number**) (в моем примере классов всего 3 на выходе), и производит оптимизацию над картинкой так чтобы было верным условие:

``torch.argmax(model(image)) == class_number``

Для этого нужно задать условие на функцию потерь. Это уже часть вашего задания понять как это сделать.

В своем примере я покажу как оптимизировать картинку, чтобы количество 

In [3]:
def optimize_image_by_class(class_number, num_steps=50000):
    # создаем тензор, который будем оптимизировать
    image = torch.randn((1,28,28))*0.001
    image = image.to(device)
    image.requires_grad = True
    
    # задайте оптимизатор
    # вы можете выбрать какой вам больше нравится
    optim = torch.optim.Adam([image], lr=1e-4)
    
    # задайте лосс функцию соответствующую заданию
    # лосс должен зависесть от class_number
    loss = ...
    grads = []
    losses = []
    for i in tqdm(range(num_steps)):
        # здесь в цикле оптимизируется картинка
        # в оптимизатор передается ваш лосс и выход из модели
        # не забывайте обнулять градиенты с optim.zero_grad()
        # также полезно сохранять значения лосс функции и нормы градиента на каждом шаге
        # чтобы было видно как меняются эти значения в процессе "обучения" картинки.
        ...
    
    # возвращаете картинку, grads, losses

Все те же шаги было бы интересно повторить уже не со свёрточным классификатором на MNIST, а со свёрточным автоэнкодером.